# Examining Racial Discrimination in the US Job Market

### Background
Racial discrimination continues to be pervasive in cultures throughout the world. Researchers examined the level of racial discrimination in the United States labor market by randomly assigning identical résumés to black-sounding or white-sounding names and observing the impact on requests for interviews from employers.

### Data
In the dataset provided, each row represents a resume. The 'race' column has two values, 'b' and 'w', indicating black-sounding and white-sounding. The column 'call' has two values, 1 and 0, indicating whether the resume received a call from employers or not.

Note that the 'b' and 'w' values in race are assigned randomly to the resumes when presented to the employer.

### Exercises
You will perform a statistical analysis to establish whether race has a significant impact on the rate of callbacks for resumes.

Answer the following questions **in this notebook below and submit to your Github account**. 

   1. What test is appropriate for this problem? Does CLT apply?
   2. What are the null and alternate hypotheses?
   3. Compute margin of error, confidence interval, and p-value. Try using both the bootstrapping and the frequentist statistical approaches.
   4. Write a story describing the statistical significance in the context or the original problem.
   5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

You can include written notes in notebook cells using Markdown: 
   - In the control panel at the top, choose Cell > Cell Type > Markdown
   - Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet

#### Resources
+ Experiment information and data source: http://www.povertyactionlab.org/evaluation/discrimination-job-market-united-states
+ Scipy statistical methods: http://docs.scipy.org/doc/scipy/reference/stats.html 
+ Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet
+ Formulas for the Bernoulli distribution: https://en.wikipedia.org/wiki/Bernoulli_distribution

In [1]:
import pandas as pd
import numpy as np
from scipy import stats

In [2]:
data = pd.io.stata.read_stata('data/us_job_market_discrimination.dta')

In [8]:
# number of callbacks for black-sounding names
sum(data[data.race=='b'].call)

157.0

In [4]:
data.head()

,id,ad,education,ofjobs,yearsexp,honors,volunteer,military,empholes,occupspecific,...,compreq,orgreq,manuf,transcom,bankreal,trade,busservice,othservice,missind,ownership
0,b,1,4,2,6,0,0,0,1,17,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
1,b,1,3,3,6,0,1,1,0,316,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
2,b,1,4,1,6,0,0,0,0,19,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
3,b,1,3,4,6,0,1,0,1,313,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
4,b,1,3,3,22,0,0,0,0,313,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,Nonprofit


In [7]:
# Get number of rows
rows, cols = data.shape
print('Data contains {0} rows.'.format(rows))

Data contains 4870 rows.


In [26]:
# Get sample proportion of blacks
n_b = np.sum(data.race == 'b')
n_success_b = int(np.sum(data[data.race=='b'].call))
n_fail_b = int(n_b - n_success_b)
p_b = n_success_b/n_b
print('Sample has {0} successes and {1} failures for blacks.'.format(n_success_b, n_fail_b))
print('Sample proportion of success for blacks = {:2.2%}'.format(p_b))

Sample has 157 successes and 2278 failures for blacks.
Sample proportion of success for blacks = 6.45%


In [27]:
# Get sample proportion of blacks
n_w = np.sum(data.race == 'w')
n_success_w = int(np.sum(data[data.race=='w'].call))
n_fail_w = int(n_w - n_success_w)
p_w = n_success_w/n_w
print('Sample has {0} successes and {1} failures for whites.'.format(n_success_w, n_fail_w))
print('Sample proportion of success for whites = {:2.2%}'.format(p_w))

Sample has 235 successes and 2200 failures for whites.
Sample proportion of success for whites = 9.65%


## Hypothesis testing for difference of proportions

The data provided is a randomly collected sample of size $n = 4870$, with a sample proportion of success of getting a call among blacks is $\hat{p_{b}} = 6.45$% and among whites is $\hat{p_{w}} = 9.65$%. Given this data, the goal is to draw inferences on the differences of proportions among general population. 

Sample proportions are well characterized by a nearly normal distribution when certain conditions are satisﬁed, making it possible to employ the usual conﬁdence interval and hypothesis testing tools. It follows that if both sampling distributions for  sample proportions are normal, then the sampling distribution of their difference is also normal.

We can use Central Limit Theorem for Hypothesis testing if the following conditions are met for each sample proportion:
1. Randomness: Data comes from a simple random sample.
2. Independence: Each observation will not influence another as the resumes are nearly identical. The sample is less than 10% of population.
3. Success-Failure Condition: At least 10 success and 10 failures are seen in the sample, which is true, as there are $157$ successes and $2278$ failures for blacks and $235$ successes and $2200$ failures for whites respectively.
4. Independence between groups: Since a person can either be black or white and not both, we can assume independence among groups.

As all conditions are met, *Central Limit Theorem applies* with following hypothesis:
- *Null Hypothesis*: Population proportions are same.
> $p_{w} - p_{b} = 0$
- *Alternative Hypothesis*: Population proportions are different.
> $p_{w} - p_{b} \neq 0$

where $p_{w}$ and $p_{b}$ are population propertions for whites and blacks respectively. We pick a significance level of $5$% for this test to construct a $95$% confidence interval.

In [37]:
w = data[data.race=='w']
b = data[data.race=='b']

### Parametric Hypothesis testing 

In [34]:
from scipy.stats import norm

# Standard Error
var_w = p_w*(1-p_w)
var_b = p_b*(1-p_b)
se = np.sqrt(var_w/n_w + var_b/n_b)
print('Standard Error={0:0.3f}'.format(se))

# Point Estimate
pe = p_w - p_b
print('Point Estimate={0:0.3f}'.format(pe))

# Z Score
z_score = pe / se
p_val = 2*(1 - norm.cdf(z_score))
print('Z-Score={0:0.3f}, P-Value={1:0.3%}'.format(z_score, p_val))

# Margin of Error for 95% CI
me = norm.ppf(0.975) * se
print('Margin of Error={0:0.3f}, 95% Confidence Interval=({1:0.3f},{2:0.3f})'.format(me, pe-me, pe+me))

Standard Error=0.008
Point Estimate=0.032
Z-Score=4.116, P-Value=0.004%
Margin of Error=0.015, 95% Confidence Interval=(0.017,0.047)


In [25]:
# Directly using statsmodel
from statsmodels.stats.proportion import proportions_ztest
stat, p_val = proportions_ztest([n_success_w, n_success_b], [n_w, n_b])
print('Z-Score={0:0.3f}, P-Value={1:0.3%}'.format(stat, p_val))

Z-Score=4.108, P-Value=0.004%


In [35]:
# Directly using statsmodel (chi-squared)
from statsmodels.stats.proportion import proportions_chisquare
stat, p_val, tbl = proportions_chisquare([n_success_w, n_success_b], [n_w, n_b])
print('Z-Score={0:0.3f}, P-Value={1:0.3%}'.format(stat, p_val))

Z-Score=16.879, P-Value=0.004%


### Bootstrap Hypothesis testing

Bootstrap testing is accomplished using permutation, the call success data is randomly shuffled between two groups and p-value is computed as proportion of samples as extreme as the observed one.

In [55]:
# Permutation Testing
def permutation_sample(data1, data2):
    """Draws a permuted sample from data1 and data2."""
    
    x = np.concatenate((data1, data2))
    x_p = np.random.permutation(x)
    d_1 = x_p[:len(data1)]
    d_2 = x_p[len(data1):]
    
    return d_1, d_2

def draw_perm_reps(data1, data2, func, size=1):
    """Computes test statistic func size times from random permutations of data1, data2 datasets."""
    
    p_r = np.empty(size)
    for i in range(size):
        d_1, d_2 = permutation_sample(data1, data2)
        p_r[i] = func(d_1, d_2)
    
    return p_r

def diff_of_proportions(data1, data2):
    """Computes a difference of proportions between two datasets for Bernoulli trials."""
    return abs(np.sum(data1) / len(data1) - np.sum(data2)/ len(data2))

w_call = w.call.astype(int).values
b_call = b.call.astype(int).values

# Draw permutation samples
num_trials=100
p_bs = draw_perm_reps(w_call, b_call, diff_of_proportions, num_trials)

# P-Value
p_val = np.sum(p_bs >= abs(p_w - p_b)) / len(p_bs)
print('P-Value={0:0.3%}'.format(p_val))

P-Value=0.000%


*Since p-value is less than the significance level of $5$%, the difference in sample proportions is significantly different.*

## Conclusion

Based on both simulations and parametric testing, the null hypothesis is rejected and we can say with 95% confidence that the population proportions for the *two groups are significantly different, and whites are 1.7% to 4.7% more likely to get calls over blacks.*

This analysis only compared effect of race on the proportion of successful calls. However, for finding out which factor is most significant, the following methods could be used:
- Perform Logistic Regression analysis on the dataset and look at p-values for the co-efficients. This approach will be affected by collinearity, in which case, groups of independent variables should be tested together to remove collinearity.
- Look at Pearson correlation coefficients between independent and dependent variables. Again remove or combine highly correlated features for removing collinearity.